<a href="https://colab.research.google.com/github/theochemtheo/leggi/blob/main/leggi_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade xlrd
!pip install skimpy
!pip install --upgrade pandas_profiling

In [4]:
import pandas as pd
import numpy as np
import re
from typing import List
from skimpy import skim
from pandas_profiling import ProfileReport
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [2]:
urls = {
    2019: 'https://data.london.gov.uk/download/leggi/58b05a7a-14e1-48cb-b730-0f4c7e84855f/LEGGI%202019%20Final.xlsx',
    2018: 'https://data.london.gov.uk/download/leggi/e9149a66-22c7-4e4d-8f7c-a7ffb779a2e8/LEGGI_2018_FINAL.xlsx',
    2017: 'https://data.london.gov.uk/download/leggi/4b951dff-7512-4ad1-af97-b6498460596d/LEGGI_2017.xls',
    2016: 'https://data.london.gov.uk/download/leggi/10bce2e1-7d9d-4fbf-bb1a-f1341ef247b2/LEGGI_2016.xls',
    2015: 'https://data.london.gov.uk/download/leggi/d47e3a93-62e5-4de2-b838-56d966e8cc39/LEGGI_2015.xls',
    2014: 'https://data.london.gov.uk/download/leggi/641c64dd-2d6a-4396-8bb5-b2c875a3ec83/LEGGI_2014_confirmed_v2.xls',
    2013: 'https://data.london.gov.uk/download/leggi/19f42926-b697-4838-b8cf-d4d311a5efb8/LEGGI_2013_final_Jun_2016.xls',
    2012: 'https://data.london.gov.uk/download/leggi/20f820ff-bdfd-4863-8db5-7ef9ac53c266/LEGGI_2012_v1_publishedDS%202014.xls',
    2011: 'https://data.london.gov.uk/download/leggi/9ca1dc79-5e81-4380-956f-15b2c348944b/LEGGI_2011_V2_publishedDS_2014.xls',
    2010: 'https://data.london.gov.uk/download/leggi/3c94a4fe-960a-4c8c-96d8-1d8febc47c7b/LEGGI_2010_v2_publishedDS_2014.xls'
}

In [3]:
energy = {}
ghg = {}
for year, url in urls.items():
  with pd.ExcelFile(url) as workbook:
    # naming scheme changed in 2019
    energy_sheet = '01 Energy'
    ghg_sheet = '02 CO2'
    if year == 2019:
      energy_sheet = 'Borough kWh'
      ghg_sheet = 'Borough GHG'
    energy[year] = pd.read_excel(workbook, energy_sheet)
    ghg[year] = pd.read_excel(workbook, ghg_sheet)

In [6]:
ghg[2015]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,Table 2.0,"Greenhouse gas emissions by borough, 2015",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Total CO2/ CO2e emissions (kt),Sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grand Total
3,NaN,NaN,Domestic (CO2e),NaN,NaN,NaN,NaN,Industrial and Commercial (CO2e),NaN,NaN,...,NaN,NaN,NaN,Transport (CO2e or CO2),NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Electricity,Gas,Coal,Oil,Total,Electricity,Electricity w/o rail,Gas,...,Oil,Waste and Renewables,Total,Aviation,Shipping,Railways_ diesel,Railways_ electric,Road Transport,Total,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,NaN,Waltham Forest,124.323753,209.598601,1.878529,0.741908,336.542791,120.180528,109.140106,50.502444,...,9.666667,7.146898,176.522304,0.662228,0,2.101147,12.16646,175.878146,190.80798,703.873076
80,NaN,Wandsworth,174.826496,267.741608,1.347738,1.428659,445.344501,167.834952,134.057564,108.476282,...,6.267318,5.257104,254.084961,13.527938,0.111696,1.456242,37.22242,169.091776,221.410072,920.839534
81,NaN,Westminster,170.210111,169.235956,0.393425,3.038072,342.877564,1141.62682,1086.950662,437.6169,...,47.559508,1.362528,1581.970742,0,1.740475,2.974163,60.252704,255.398694,320.366035,2245.214341
82,NaN,Grand Total (kt),4547.243775,7056.636922,43.160584,48.296858,11695.338139,9098.249491,8419.65195,3572.334934,...,601.443045,183.582054,12794.940792,942.846061,31.806999,83.949808,747.809238,6432.16196,8238.574066,32728.852998


In [7]:
def _clean_text(string: str) -> pd.DataFrame:
  string = string.strip().lower().replace(' ', '_')
  string = re.sub(r'[*()/\-&]', '', string)
  return string

def clean_df_text(df: pd.DataFrame) -> pd.DataFrame:
  return df.applymap(lambda s: _clean_text(s) if type(s) == str else s)

In [8]:
def remove_total_columns(df: pd.DataFrame,
                         header_size: int = 3) -> pd.DataFrame:
  totals = df[:header_size].applymap(lambda s: bool(re.search('total', s)) if type(s) == str else False)
  total_cols = np.unique(np.where(totals)[1])
  df = df.drop(df.columns[total_cols], axis=1)
  return df

In [9]:
def remove_bad_rows(df: pd.DataFrame,
                    bad_strings: List[str],
                    header_size: int = 3) -> pd.DataFrame:
  bads = df[header_size:].applymap(lambda s: bool(re.search(f"({'|'.join(bad_strings)})", s)) if type(s) == str else False)
  bad_rows = np.unique(np.where(bads)[0]) + header_size
  df = df.drop(df.index[bad_rows], axis=0)
  return df

In [10]:
def crop_sheet(df: pd.DataFrame, data_type: str, year: int) -> pd.DataFrame:
  # different layout pre 2013:
  if year > 2012:
    top_left = np.where(df == 'year')
    offset = 2
  else:
    top_left = np.where(df == 'borough')
    offset = 1
  # pre-2019 ghg emissions sheet inexplicably has a hidden second table
  if data_type == 'ghg' and year < 2019:
    cutoff_row = np.where(df == 'table_2.3:')[0][0] - 1
    df = df.iloc[top_left[0][0] - offset:cutoff_row, top_left[1][0]:]
  else:
    df = df.iloc[top_left[0][0] - offset:, top_left[1][0]:]
  return df

In [11]:
def fix_heading(df: pd.DataFrame) -> pd.DataFrame:
  new_header = df[:3].fillna(axis=0, method='ffill')
  new_header = new_header.fillna(axis=1, method='ffill')
  df[:3] = new_header
  return df

In [12]:
def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
  new_names = df.iloc[:2, 1:].agg('_'.join, axis=0)
  new_names = ['borough', *new_names]
  new_names = [col.replace('__', '_') for col in new_names]
  new_names = [col.replace('from_decc_modelling', 'co2e') for col in new_names]
  new_names = [col.replace('co2e_or_co2', 'co2e') for col in new_names]
  df.columns = new_names
  df = df[3:]
  return df

In [13]:
def drop_unreliable_columns(df: pd.DataFrame, bad_column_prefixes: List[str]) -> pd.DataFrame:
  bad_columns = []
  for prefix in bad_column_prefixes:
    bad_columns += [col for col in df.columns if prefix in col]
  return df.drop(columns=bad_columns)

In [14]:
def wide_to_narrow(df: pd.DataFrame) -> pd.DataFrame:
  narrow = []
  for sector in ['domestic', 'industrial_and_commercial']:
    cols = df.columns[[sector in col for col in df.columns]]
    renamed = [col.replace(f'{sector}_', '') for col in cols]
    df = df.rename(columns=dict(zip(cols, renamed)))
    sector_df = df.melt(value_vars=renamed, var_name='source', ignore_index=False)
    sector_df = sector_df.reset_index()
    sector_df['sector'] = sector
    narrow += [sector_df]
  df = pd.concat(narrow, ignore_index=True)
  return df

In [15]:
def tidy_sheet(df: pd.DataFrame,
               data_type: str,
               year: int,
               bad_rows: List[str],
               bad_column_prefixes: List[str]) -> pd.DataFrame:
  df.columns = range(df.columns.size)
  df = clean_df_text(df)
  df = crop_sheet(df, data_type, year)
  df = fix_heading(df)
  df = remove_total_columns(df)
  df = remove_bad_rows(df, bad_rows)
  df = df.dropna(how='all', axis=0)
  df = rename_columns(df)
  df = drop_unreliable_columns(df, bad_column_prefixes)
  df = df.reset_index(drop=True)
  df = df.set_index('borough')
  if data_type == 'ghg':
    df.columns = df.columns.str.replace('_co2e', '')
  df = wide_to_narrow(df)
  df['type'] = data_type
  df['year'] = year
  df = df[['borough', 'year', 'type', 'sector', 'source', 'value']]
  df = df.astype({'borough': 'category',
                  'type': 'category',
                  'sector': 'category',
                  'source': 'category',
                  'value': float})
  return df

In [16]:
bad_rows = ['total', 'not_grid_connected', 'unapportioned', 'table_2.3:', 'excl._lucluf', 'large_gas_users']
bad_column_prefixes = ['transport', 'nrmm', 'ippu', 'waste', 'fugitive', 'afolu', 'agriculture', 'electricity_wo_rail']
tidy_sheet(ghg[2015], 'ghg', 2015, bad_rows, bad_column_prefixes)

,borough,year,type,sector,source,value
0,barking_and_dagenham,2015,ghg,domestic,electricity,103.589404
1,barnet,2015,ghg,domestic,electricity,242.089762
2,bexley,2015,ghg,domestic,electricity,148.541005
3,brent,2015,ghg,domestic,electricity,165.292744
4,bromley,2015,ghg,domestic,electricity,217.802587
...,...,...,...,...,...,...
391,sutton,2015,ghg,industrial_and_commercial,oil,8.547695
392,tower_hamlets,2015,ghg,industrial_and_commercial,oil,18.742878
393,waltham_forest,2015,ghg,industrial_and_commercial,oil,9.700659
394,wandsworth,2015,ghg,industrial_and_commercial,oil,6.289357


In [17]:
tidy_ghg = {year: tidy_sheet(sheet, 'ghg', year, bad_rows, bad_column_prefixes) for year, sheet in ghg.items()}

In [18]:
tidy_energy = {year: tidy_sheet(sheet, 'energy', year, bad_rows, bad_column_prefixes) for year, sheet in energy.items()}

In [19]:
data = pd.concat([*tidy_ghg.values(), *tidy_energy.values()])

In [20]:
data

,borough,year,type,sector,source,value
0,barking_and_dagenham,2019,ghg,domestic,electricity,5.804402e+01
1,barnet,2019,ghg,domestic,electricity,1.413671e+02
2,bexley,2019,ghg,domestic,electricity,8.340552e+01
3,brent,2019,ghg,domestic,electricity,9.606731e+01
4,bromley,2019,ghg,domestic,electricity,1.230897e+02
...,...,...,...,...,...,...
391,sutton,2010,energy,industrial_and_commercial,oil,5.152090e+07
392,tower_hamlets,2010,energy,industrial_and_commercial,oil,3.361070e+07
393,waltham_forest,2010,energy,industrial_and_commercial,oil,5.412602e+07
394,wandsworth,2010,energy,industrial_and_commercial,oil,2.794689e+07


In [21]:
profile = data.profile_report()

In [22]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
data.to_hdf('leggi_2010-2019_tidy.h5', key='df', format='table')